In [1]:
import numpy as np
# import cupy as cp
# print(cp.__version__)
import tifffile as tiff
from skimage.morphology import skeletonize  # Import for skeletonization
from skan import Skeleton, summarize
from joblib import Parallel, delayed
import sys

In [2]:
def print_branches(skeleton_cleaned):
    skel = Skeleton(skeleton_cleaned)
    branch_data = summarize(skel)

    branch_type_0 = branch_data[branch_data['branch-type'] == 0]
    branch_type_1 = branch_data[branch_data['branch-type'] == 1]
    branch_type_2 = branch_data[branch_data['branch-type'] == 2]
    branch_type_3 = branch_data[branch_data['branch-type'] == 3]

    print(f"Number of branch type 0: {len(branch_type_0)}")
    print(f"Number of branch type 1: {len(branch_type_1)}")
    print(f"Number of branch type 2: {len(branch_type_2)}")
    print(f"Number of branch type 3: {len(branch_type_3)}")

In [3]:
# binary_volume = tiff.imread("/zhome/57/0/203104/specialCourse_fall24/outputs/output_volume.tif")
binary_volume = tiff.imread("C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/skeleleton_clean_x20_th5_UNET_TOP.tif")
# Get the size of the volume
print(f'Size of the volume is: {binary_volume.shape}')

Size of the volume is: (400, 200, 200)


In [4]:
print_branches(binary_volume)

Number of branch type 0: 75
Number of branch type 1: 33
Number of branch type 2: 29
Number of branch type 3: 2


In [5]:
skel = Skeleton(binary_volume)
branch_data = summarize(skel)

In [6]:
import cc3d
import numpy as np

labels_out = cc3d.connected_components(binary_volume)
labels_out, N = cc3d.connected_components(binary_volume, return_N=True)
print(N)

104


In [7]:
# Initialize a dictionary to map each branch ID to its corresponding blob(s)
branch_blob_mapping = {}
pixel_blob_mapping = np.zeros_like(labels_out, dtype=int)

# Step 2: For each branch, get the coordinates and map them to the corresponding blob label(s)
for branch_id in branch_data.index:
    # Get the coordinates of the current branch in the skeleton structure
    coordinates = skel.path_coordinates(branch_id)
    
    # Fetch the labels at those coordinates from the labels_out array
    branch_labels = labels_out[coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]]
    
    # Find the unique label(s) associated with this branch
    unique_blob_labels = np.unique(branch_labels)
    
    # Store the unique label(s) in the branch_blob_mapping
    if len(unique_blob_labels) > 1:
        branch_blob_mapping[branch_id] = unique_blob_labels
    else:
        branch_blob_mapping[branch_id] = unique_blob_labels[0]
    
    # Step 3: Map each pixel in this branch to its corresponding blob label(s)
    for i, coord in enumerate(coordinates):
        pixel_blob_mapping[tuple(coord)] = branch_labels[i]


In [8]:
def Bresenham3D(x1, y1, z1, x2, y2, z2):
    ListOfPoints = []
    ListOfPoints.append((x1, y1, z1))
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    dz = abs(z2 - z1)
    if (x2 > x1):
        xs = 1
    else:
        xs = -1
    if (y2 > y1):
        ys = 1
    else:
        ys = -1
    if (z2 > z1):
        zs = 1
    else:
        zs = -1
 
    # Driving axis is X-axis"
    if (dx >= dy and dx >= dz):        
        p1 = 2 * dy - dx
        p2 = 2 * dz - dx
        while (x1 != x2):
            x1 += xs
            if (p1 >= 0):
                y1 += ys
                p1 -= 2 * dx
            if (p2 >= 0):
                z1 += zs
                p2 -= 2 * dx
            p1 += 2 * dy
            p2 += 2 * dz
            ListOfPoints.append((x1, y1, z1))
 
    # Driving axis is Y-axis"
    elif (dy >= dx and dy >= dz):       
        p1 = 2 * dx - dy
        p2 = 2 * dz - dy
        while (y1 != y2):
            y1 += ys
            if (p1 >= 0):
                x1 += xs
                p1 -= 2 * dy
            if (p2 >= 0):
                z1 += zs
                p2 -= 2 * dy
            p1 += 2 * dx
            p2 += 2 * dz
            ListOfPoints.append((x1, y1, z1))
 
    # Driving axis is Z-axis"
    else:        
        p1 = 2 * dy - dz
        p2 = 2 * dx - dz
        while (z1 != z2):
            z1 += zs
            if (p1 >= 0):
                y1 += ys
                p1 -= 2 * dz
            if (p2 >= 0):
                x1 += xs
                p2 -= 2 * dz
            p1 += 2 * dy
            p2 += 2 * dx
            ListOfPoints.append((x1, y1, z1))
    return ListOfPoints

In [9]:
import numpy as np
from scipy.spatial.distance import cdist

# Paso 1: Calcular centroides de los blobs en 3D
def calculate_centroid(blob_labels, labels_out):
    centroids = {}
    for blob in np.unique(blob_labels):
        # Encuentra las coordenadas de todos los píxeles de ese blob
        coords = np.array(np.where(labels_out == blob)).T
        # Calcula el centroide como el promedio de las coordenadas
        centroid = np.mean(coords, axis=0)
        centroids[blob] = centroid
    return centroids

# Paso 2: Encontrar los blobs más cercanos en 3D
def find_closest_blobs(centroids):
    blobs = list(centroids.keys())
    distances = cdist([centroids[blob] for blob in blobs], [centroids[blob] for blob in blobs])
    np.fill_diagonal(distances, np.inf)  # Evitar que los blobs se conecten consigo mismos
    closest_pairs = np.unravel_index(np.argmin(distances), distances.shape)
    return blobs[closest_pairs[0]], blobs[closest_pairs[1]]

# Paso 3: Conectar los blobs más cercanos con una línea recta 3D
def connect_blobs_3d(labels_out, pixel_blob_mapping, skeleton_cleaned):
    import numpy as np
    from scipy.spatial import distance

    # Copia del esqueleto para modificaciones
    skel = np.copy(skeleton_cleaned)

    # Calcular centroides de todos los blobs
    centroids = calculate_centroid(labels_out, labels_out)

    # Encontrar el par de blobs más cercanos
    blob1, blob2 = find_closest_blobs(centroids)
    print(f"Connecting blobs {blob1} and {blob2}")

    # Coordenadas de todos los puntos en cada blob
    coords_blob1 = np.array(np.where(labels_out == blob1)).T
    coords_blob2 = np.array(np.where(labels_out == blob2)).T

    # Calcular las distancias entre cada par de puntos
    distances = distance.cdist(coords_blob1, coords_blob2, 'euclidean')
    min_idx = np.unravel_index(np.argmin(distances), distances.shape)

    # Seleccionar los puntos más cercanos
    p1 = coords_blob1[min_idx[0]]
    p2 = coords_blob2[min_idx[1]]

    print(f"Closest points: {p1} in blob {blob1}, {p2} in blob {blob2}")

    # Calcular la línea de Bresenham entre los puntos (en 3D)
    line_points = Bresenham3D(p1[0], p1[1], p1[2], p2[0], p2[1], p2[2])

    # Conectar los blobs en el mapeo de píxeles
    for point in line_points:
        pixel_blob_mapping[tuple(point)] = blob1  # O usar una mezcla de ambos blobs
        skel[tuple(point)] = 1
    
    return skel


In [10]:
binary_volume_copy = np.copy(binary_volume)
while True:
    # Paso 4: Repetir los pasos 2 y 3 hasta que todos los blobs estén conectados
    binary_volume_copy = connect_blobs_3d(labels_out, pixel_blob_mapping, binary_volume_copy)
    
    # Free up memory by deleting unused variables
    del labels_out, pixel_blob_mapping, branch_blob_mapping
    
    labels_out, N = cc3d.connected_components(binary_volume_copy, return_N=True)
    print(f"Number of blobs: {N}")
    
    # Reinitialize pixel_blob_mapping to reduce memory usage
    # Initialize a dictionary to map each branch ID to its corresponding blob(s)
    branch_blob_mapping = {}
    pixel_blob_mapping = np.zeros_like(labels_out, dtype=int)

    skel = Skeleton(binary_volume_copy)
    branch_data = summarize(skel)

    # Step 2: For each branch, get the coordinates and map them to the corresponding blob label(s)
    for branch_id in branch_data.index:
        # Get the coordinates of the current branch in the skeleton structure
        coordinates = skel.path_coordinates(branch_id)
        
        # Fetch the labels at those coordinates from the labels_out array
        branch_labels = labels_out[coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]]
        
        # Find the unique label(s) associated with this branch
        unique_blob_labels = np.unique(branch_labels)
        
        # Store the unique label(s) in the branch_blob_mapping
        if len(unique_blob_labels) > 1:
            branch_blob_mapping[branch_id] = unique_blob_labels
        else:
            branch_blob_mapping[branch_id] = unique_blob_labels[0]
        
        # Step 3: Map each pixel in this branch to its corresponding blob label(s)
        for i, coord in enumerate(coordinates):
            pixel_blob_mapping[tuple(coord)] = branch_labels[i]
    
    if N == 3:
        print("All blobs are connected!")
        break

Connecting blobs 63 and 64
Closest points: [328  73  74] in blob 63, [329  73  76] in blob 64
Number of blobs: 103
Connecting blobs 81 and 82
Closest points: [357 127  45] in blob 81, [360 127  45] in blob 82
Number of blobs: 102
Connecting blobs 27 and 28
Closest points: [262  88 192] in blob 27, [263  86 193] in blob 28
Number of blobs: 101
Connecting blobs 41 and 42
Closest points: [300 167 106] in blob 41, [300 169 103] in blob 42
Number of blobs: 100
Connecting blobs 91 and 92
Closest points: [381 113   6] in blob 91, [384 114   8] in blob 92
Number of blobs: 99
Connecting blobs 57 and 60
Closest points: [324  64  93] in blob 57, [327  65  94] in blob 60
Number of blobs: 98
Connecting blobs 80 and 84
Closest points: [366 123  45] in blob 80, [368 122  44] in blob 84
Number of blobs: 97
Connecting blobs 35 and 36
Closest points: [291 118 141] in blob 35, [293 118 142] in blob 36
Number of blobs: 96
Connecting blobs 81 and 85
Closest points: [367  55  75] in blob 81, [372  55  74] i

KeyboardInterrupt: 

In [ ]:
# blob_mapping_connected = connect_blobs_3d(labels_out, pixel_blob_mapping, binary_volume)
tiff.imwrite('binary_volume_copy_UNET_TOP.tif', binary_volume_copy)
